In [244]:
from lxml import html
from lxml.html.clean import clean_html
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [245]:
# Download and output raw HTML
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
page = requests.get(url)
tree = html.fromstring(page.content)
#click.echo(print(soup))

In [246]:
from lxml.html.clean import Cleaner


In [332]:
def fetch_html(url, clean=True):
    page = requests.get(url)
    html = page.content.decode("utf-8", "ignore")
    if clean:
        cleaner = Cleaner(style=True, links=True, add_nofollow=True,
                          page_structure=False, safe_attrs_only=True)
        return cleaner.clean_html(html)
    return html

def parse_table(html):
    soup = BeautifulSoup(html)
    return pd.DataFrame(
        data=[[[(td.text
                  .replace('\n', '')
                  .replace('\xa0', '')
                  .replace('km2', '')
                  .replace('sqmi', '')
                  .split(' / ')[-1]
                  .split('[')[0]
                  .split('\ufeff ')[0]
                  .split('♠')[-1]
                  .strip('%|\/'))
              for td in tr.findAll('td')]
             for tr in table.findAll('tr')]
            for table in soup.findAll('table')][4:][0],
        columns=['rank', 'city', 'state', 'pop_estimate', 'census', 
                 'pop_delta', 'pop_density__mi', 'pop_density__km2', 
             'land_area__mi', 'land_area__km2', 'location'])[1:]

def clean_table(table):
    return (
        table.assign(
            pop_estimate=table.pop_estimate.str.replace(',','').astype(int),
            census=table.census.str.replace(',','').astype(float),
            pop_delta=table.pop_delta.str.replace('+','').astype(float, errors='ignore'),
            pop_density__mi=table.pop_density__mi.str.replace(',','').astype(float, errors='ignore'),
            pop_density__km2=table.pop_density__km2.str.replace(',','').astype(float, errors='ignore'),
            land_area__mi=table.land_area__mi.str.replace(',','').astype(int, errors='ignore'),
            land_area__km2=table.land_area__km2.str.replace(',','').astype(int, errors='ignore'),
            latitude=table.location.str.split(';').str.get(0),
            longitude=table.location.str.split(';').str.get(1),
            location=table.city + ', ' + table.state,
            city=table.city,
            state=table.state)
        .set_index('location', drop=True))

In [334]:
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
page = fetch_html(url)
table = parse_table(page).pipe(clean_table)
table.head()

,rank,city,state,pop_estimate,census,pop_delta,pop_density__mi,pop_density__km2,land_area__mi,land_area__km2,latitude,longitude
location,,,,,,,,,,,,
"New York, New York",1,New York,New York,8622698,8175133.0,5.47,301.5,780.9,28317,10933,40.6635,-73.9387
"Los Angeles, California",2,Los Angeles,California,3999759,3792621.0,5.46,468.7,1213.9,8484,3276,34.0194,-118.4108
"Chicago, Illinois",3,Chicago,Illinois,2716450,2695598.0,0.77,227.3,588.7,11900,4600,41.8376,-87.6818
"Houston, Texas",4,Houston,Texas,2312717,2100263.0,10.12,637.5,1651.1,3613,1395,29.7866,-95.3909
"Phoenix, Arizona",5,Phoenix,Arizona,1626078,1445632.0,12.48,517.6,1340.6,3120,1200,33.5722,-112.0901


In [249]:
z = [
    [
        [
            td.text.replace('\n', '')
            for td in tr.findAll('td')]
        for tr in table.findAll('tr')
        if len(tr) > 10
    
    ]
    for table in soup.findAll('table')
    if len(table.findAll('tr')) > 25
]
len(z)
z[0]

[[],
 ['1',
  'New York[6]',
  '\xa0New York',
  '8,622,698',
  '8,175,133',
  '+5.47%',
  '301.5\xa0sq\xa0mi',
  '780.9\xa0km2',
  '28,317/sq\xa0mi',
  '10,933/km2',
  '40°39′49″N 73°56′19″W\ufeff / \ufeff40.6635°N 73.9387°W\ufeff / 40.6635; -73.9387\ufeff (1 New York City)'],
 ['2',
  'Los Angeles',
  '\xa0California',
  '3,999,759',
  '3,792,621',
  '+5.46%',
  '468.7\xa0sq\xa0mi',
  '1,213.9\xa0km2',
  '8,484/sq\xa0mi',
  '3,276/km2',
  '34°01′10″N 118°24′39″W\ufeff / \ufeff34.0194°N 118.4108°W\ufeff / 34.0194; -118.4108\ufeff (2 Los Angeles)'],
 ['3',
  'Chicago',
  '\xa0Illinois',
  '2,716,450',
  '2,695,598',
  '+0.77%',
  '227.3\xa0sq\xa0mi',
  '588.7\xa0km2',
  '11,900/sq\xa0mi',
  '4,600/km2',
  '41°50′15″N 87°40′54″W\ufeff / \ufeff41.8376°N 87.6818°W\ufeff / 41.8376; -87.6818\ufeff (3 Chicago)'],
 ['4',
  'Houston[7]',
  '\xa0Texas',
  '2,312,717',
  '2,100,263',
  '+10.12%',
  '637.5\xa0sq\xa0mi',
  '1,651.1\xa0km2',
  '3,613/sq\xa0mi',
  '1,395/km2',
  '29°47′12″N 95°23′27

In [331]:
table = 



,rank,city,state,pop_estimate,census,pop_delta,pop_density__mi,pop_density__km2,land_area__mi,land_area__km2,latitude,longitude
location,,,,,,,,,,,,
"New York, New York",1,New York,New York,8622698,8175133.0,5.47,301.5,780.9,28317,10933,40.6635,-73.9387
"Los Angeles, California",2,Los Angeles,California,3999759,3792621.0,5.46,468.7,1213.9,8484,3276,34.0194,-118.4108
"Chicago, Illinois",3,Chicago,Illinois,2716450,2695598.0,0.77,227.3,588.7,11900,4600,41.8376,-87.6818
"Houston, Texas",4,Houston,Texas,2312717,2100263.0,10.12,637.5,1651.1,3613,1395,29.7866,-95.3909
"Phoenix, Arizona",5,Phoenix,Arizona,1626078,1445632.0,12.48,517.6,1340.6,3120,1200,33.5722,-112.0901
"Philadelphia, Pennsylvania",6,Philadelphia,Pennsylvania,1580863,1526006.0,3.59,134.2,347.6,11683,4511,40.0094,-75.1333
"San Antonio, Texas",7,San Antonio,Texas,1511946,1327407.0,13.90,461.0,1194.0,3238,1250,29.4724,-98.5251
"San Diego, California",8,San Diego,California,1419516,1307402.0,8.58,325.2,842.3,4325,1670,32.8153,-117.1350
"Dallas, Texas",9,Dallas,Texas,1341075,1197816.0,11.96,340.9,882.9,3866,1493,32.7933,-96.7665


In [330]:
table

,rank,city,state,pop_estimate,census,pop_delta,pop_density__mi,pop_density__km2,land_area__mi,land_area__km2,latitude,longitude
location,,,,,,,,,,,,
"New York, New York",1,New York,New York,8622698,8175133.0,5.47,301.5,780.9,28317,10933,40.6635,-73.9387
"Los Angeles, California",2,Los Angeles,California,3999759,3792621.0,5.46,468.7,1213.9,8484,3276,34.0194,-118.4108
"Chicago, Illinois",3,Chicago,Illinois,2716450,2695598.0,0.77,227.3,588.7,11900,4600,41.8376,-87.6818
"Houston, Texas",4,Houston,Texas,2312717,2100263.0,10.12,637.5,1651.1,3613,1395,29.7866,-95.3909
"Phoenix, Arizona",5,Phoenix,Arizona,1626078,1445632.0,12.48,517.6,1340.6,3120,1200,33.5722,-112.0901
"Philadelphia, Pennsylvania",6,Philadelphia,Pennsylvania,1580863,1526006.0,3.59,134.2,347.6,11683,4511,40.0094,-75.1333
"San Antonio, Texas",7,San Antonio,Texas,1511946,1327407.0,13.90,461.0,1194.0,3238,1250,29.4724,-98.5251
"San Diego, California",8,San Diego,California,1419516,1307402.0,8.58,325.2,842.3,4325,1670,32.8153,-117.1350
"Dallas, Texas",9,Dallas,Texas,1341075,1197816.0,11.96,340.9,882.9,3866,1493,32.7933,-96.7665


In [311]:
table.assign(
    pop_estimate=table.pop_estimate.str.replace(',','').astype(int),
    census=table.census.str.replace(',','').astype(float),
    pop_delta=table.pop_delta.str.replace('+','').astype(float, errors='ignore'),
    pop_density__mi=table.pop_density__mi.str.replace(',','').astype(float, errors='ignore'),
    pop_density__km2=table.pop_density__km2.str.replace(',','').astype(float, errors='ignore'),
    land_area__mi=table.land_area__mi.str.replace(',','').astype(int, errors='ignore'),
    land_area__km2=table.land_area__km2.str.replace(',','').astype(int, errors='ignore'),
    latitude=table.location.str.split(';').str.get(0),
    longitude=table.location.str.split(';').str.get(1),
    location=table.city + ', ' + table.state,
    city=table.city,
    state=table.state
).set_index('location', drop=True)

,rank,city,state,pop_estimate,census,pop_delta,pop_density__mi,pop_density__km2,land_area__mi,land_area__km2,latitude,longitude
location,,,,,,,,,,,,
"New York, New York",1,New York,New York,8622698,8175133.0,5.47,301.5,780.9,28317,10933,40.6635,-73.9387
"Los Angeles, California",2,Los Angeles,California,3999759,3792621.0,5.46,468.7,1213.9,8484,3276,34.0194,-118.4108
"Chicago, Illinois",3,Chicago,Illinois,2716450,2695598.0,0.77,227.3,588.7,11900,4600,41.8376,-87.6818
"Houston, Texas",4,Houston,Texas,2312717,2100263.0,10.12,637.5,1651.1,3613,1395,29.7866,-95.3909
"Phoenix, Arizona",5,Phoenix,Arizona,1626078,1445632.0,12.48,517.6,1340.6,3120,1200,33.5722,-112.0901
"Philadelphia, Pennsylvania",6,Philadelphia,Pennsylvania,1580863,1526006.0,3.59,134.2,347.6,11683,4511,40.0094,-75.1333
"San Antonio, Texas",7,San Antonio,Texas,1511946,1327407.0,13.90,461.0,1194.0,3238,1250,29.4724,-98.5251
"San Diego, California",8,San Diego,California,1419516,1307402.0,8.58,325.2,842.3,4325,1670,32.8153,-117.1350
"Dallas, Texas",9,Dallas,Texas,1341075,1197816.0,11.96,340.9,882.9,3866,1493,32.7933,-96.7665
